In [74]:
# importing required libraries
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import time

In [75]:
def gemSoup(url):
    try:
        page = requests.get(url)
    except Exception as exception:
        print(exception)
    page_status = page.status_code   # if page try fails then page is referenced before assignment correct it
    if page_status == 200:
        print("{} loaded successfully".format(url))
    else:
        print("{} loading failed with status code {}".format(url, page_status)) 
    encoding = page.encoding
    headers = page.headers
    # content = page.content # binary dump
    # text = page.txt # textual dump
    #print(encoding, headers)
    soup = bs(page.content,'html.parser')
    return soup

In [3]:
my_soup = gemSoup("https://bidplus.gem.gov.in/bidresultlists")

https://bidplus.gem.gov.in/bidresultlists loaded successfully


In [76]:
# GeM Bid data scaper
def gemBid(soup):
    bid_number = []
    bid_status = []
    bid_dep_add = []
    bid_iqc = []
    bid_date = []
    # load bid section
    bid_list = my_soup.findAll('div', {'id':'pagi_content'})
    column_data = my_soup.findAll('div', {'class':'col-block'})
    for element in bid_list:
        bid_numbers = element.findAll('p', {'class':'bid_no'})
        bid_status_data = element.findAll('span', {'class':'text-success'})
    for bid in bid_numbers:
        bid_number.append(bid.text)
    for status in bid_status_data:
        bid_status.append(status.text)
    iqc_start = 0 # Items, Quantity required, Corrigendum
    col_end = len(column_data)
    for index in range(iqc_start,col_end,3):
        bid_iqc.append(column_data[index].text)
    add_start = 1 # bidder address
    col_end = len(column_data)
    for index in range(add_start,col_end,3):
        bid_dep_add.append(column_data[index].text)
    bid_date_start = 2 # bidding start and end date
    col_end = len(column_data)
    for index in range(bid_date_start,col_end,3):
        bid_date.append(column_data[index].text)
    return (bid_number, bid_status, bid_dep_add, bid_iqc, bid_date) #clean all lists before returning
    

In [77]:
# pagination
def gemPage(soup):
    pagination = soup.findAll('ul', {'class':'pagination'})
    for page in pagination:
        current_page = int(page.findAll('li', {'class':'active'})[0].text)
        # last page value is coming in 4th 'a' tag
        last_page = page.findAll('a', {'data-ci-pagination-page':True})[3]['data-ci-pagination-page']
        rel = page.findAll('a', {'rel':True})
        if current_page >= 1 and page.findAll('a', {'rel':True})[0]['rel'][0] == 'next':
            next_page_ref = page.findAll('a', {'rel':True})[0]['href']
            return (current_page, last_page, next_page_ref)
        elif page.findAll('a', {'rel':True})[0]['rel'][0] == 'prev':
            print('last page reached')
            return None
        else:
            print("unknown error")
        break

In [78]:
domain = "https://bidplus.gem.gov.in"
rel = "/bidresultlists"
url = domain + rel
my_soup = gemSoup(url)
BID_NO = []
BID_STATUS = [] 
BID_D_ADD = []
BID_IQC = []
BID_DATE = []
page = gemPage(my_soup)
for page_number in range(int(page[0]) ,int(page[1]) +1):
    bid_data = gemBid(my_soup)
    BID_NO.append(bid_data[0])
    BID_STATUS.append(bid_data[1])
    BID_D_ADD.append(bid_data[2])
    BID_IQC.append(bid_data[3])
    BID_DATE.append(bid_data[4])
    try:
        url = domain + page[2]
        time.sleep(5)
        my_soup = gemSoup(url)
        page = gemPage(my_soup)
    except:
        continue
df = pd.DataFrame({'BID_NO':BID_NO,'BID_STATUS':BID_STATUS, 'BID_D_ADD':BID_D_ADD, 'BID_IQC':BID_IQC, 'BID_DATE':BID_DATE})
json_data = df.to_json('./GeM_export.json', orient='index')

https://bidplus.gem.gov.in/bidresultlists loaded successfully
https://bidplus.gem.gov.in/bidresultlists?bidresultlists&page_no=AMCR24yMNFkfoXF3wKPmGMy_wV8TJPAlxm6oWiTHGOI loaded successfully
last page reached


In [58]:
next_page = gemPage(my_soup)

In [59]:
print(next_page)

(1, '122', '/bidresultlists?bidresultlists&page_no=AMCR24yMNFkfoXF3wKPmGMy_wV8TJPAlxm6oWiTHGOI')


In [12]:
url = domain + next_page
my_soup = gemSoup(url)

https://bidplus.gem.gov.in/bidresultlists?bidresultlists&page_no=AMCR24yMNFkfoXF3wKPmGMy_wV8TJPAlxm6oWiTHGOI loaded successfully


In [13]:
gembid = gemBid(my_soup)

In [15]:
#print(gembid[0], gembid[1], gembid[2], gembid[3], gembid[4])
print(gembid[1])

['Not Evaluated', 'Financial Evaluation', 'Not Evaluated', 'Technical Evaluation', 'Bid Award', 'Not Evaluated', 'Technical Evaluation', 'Not Evaluated', 'Not Evaluated', 'Bid Award']


In [13]:
df = pd.DataFrame({'BID_NO':gembid[0],'BID_STATUS':gembid[1], 'BID_D_ADD':gembid[2], 'BID_IQC':gembid[3], 'BID_DATE':gembid[4]})

In [14]:
df.head(10)

,BID_NO,BID_STATUS,BID_D_ADD,BID_IQC,BID_DATE
0,BID NO: GEM/2022/B/2005624,Not Evaluated,\nDepartment Name And Address:\n\n ...,"\nItems: PA System,PA System,PA System,PA Syst...",\nStart Date: 03-03-2022 06:00 PM\nEnd Date: 0...
1,BID NO: GEM/2022/B/2005574,Not Evaluated,\nDepartment Name And Address:\n\n ...,"\nItems: ALS,ALS,ALS\nQuantity Required: 3\nVi...",\nStart Date: 03-03-2022 05:55 PM\nEnd Date: 0...
2,BID NO: GEM/2022/B/2005272,Not Evaluated,\nDepartment Name And Address:\n\n ...,\nItems: Printer\nQuantity Required: 1\nView C...,\nStart Date: 03-03-2022 05:23 PM\nEnd Date: 0...
3,BID NO: GEM/2022/B/2005090,Not Evaluated,\nDepartment Name And Address:\n\n ...,\nItems: Short Term Cab & Taxi Hiring Services...,\nStart Date: 03-03-2022 05:47 PM\nEnd Date: 0...
4,BID NO: GEM/2022/B/2004756,Not Evaluated,\nDepartment Name And Address:\n\n ...,\nItems: CURTAINS\nQuantity Required: 210\nVie...,\nStart Date: 03-03-2022 03:45 PM\nEnd Date: 0...
5,BID NO: GEM/2022/B/2004713,Not Evaluated,\nDepartment Name And Address:\n\n ...,\nItems: Steel Chairs\nQuantity Required: 21\n...,\nStart Date: 03-03-2022 03:46 PM\nEnd Date: 0...
6,BID NO: GEM/2022/B/2004642,Not Evaluated,\nDepartment Name And Address:\n\n ...,"\nItems: Kyocera 1800-200 drum unit change,Key...",\nStart Date: 03-03-2022 03:51 PM\nEnd Date: 0...
7,BID NO: GEM/2022/B/2004555,Not Evaluated,\nDepartment Name And Address:\n\n ...,"\nItems: Full Plate,Qtr Plate 7,Bowl 9 cm,Dong...",\nStart Date: 03-03-2022 05:39 PM\nEnd Date: 0...
8,BID NO: GEM/2022/B/2003680,Not Evaluated,\nDepartment Name And Address:\n\n ...,"\nItems: Web Camera for Video Conferencing,Mic...",\nStart Date: 03-03-2022 12:55 PM\nEnd Date: 0...
9,BID NO: GEM/2022/B/2003274,Not Evaluated,\nDepartment Name And Address:\n\n ...,"\nItems: Brake Bleeding Machine,Parts Cleaner ...",\nStart Date: 03-03-2022 12:00 PM\nEnd Date: 0...


In [15]:
df.BID_NO

0     BID NO: GEM/2022/B/2005624
1     BID NO: GEM/2022/B/2005574
2     BID NO: GEM/2022/B/2005272
3     BID NO: GEM/2022/B/2005090
4     BID NO: GEM/2022/B/2004756
5     BID NO: GEM/2022/B/2004713
6     BID NO: GEM/2022/B/2004642
7     BID NO: GEM/2022/B/2004555
8     BID NO: GEM/2022/B/2003680
9     BID NO: GEM/2022/B/2003274
Name: BID_NO, dtype: object